In [1]:
from nltk.corpus import wordnet as wn
import nltk
import csv
import time
import random

### 1. Download wordnet and files with class names of ImageNet-(2)1k 

In [2]:
_ = nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mathe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Read ids of classes in ImageNet-21k and ImageNet-1k.

In [5]:
# Read synset ids and names of classes in ImageNet-21k
synset_ids_21k = []
class_names_21k = []
with open('../classes_imagenet/classes_in_imagenet_21k.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader) # skip first line
    for row in csv_reader:
        synset_ids_21k.append(row[0])  # e.g. 'n00004475'
        class_names_21k.append(row[1].replace(' ', '_')) # e.g. 'organism'

In [6]:
# Read synset ids and names of classes in ImageNet-1k
synset_ids_1k = []
class_names_1k = []
with open('../classes_imagenet/classes_in_imagenet_1k.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    i = 0
    for row in csv_reader:
        class_id = row[0].split(':')[0]
        synset_ids_1k.append(class_id)
        class_name = row[0].split(':')[1].split(',')[0].replace(' ', '_')[1:]
        class_names_1k.append(class_name)

### 2. Generate synset_ids for out-of-distribution classes 

In [31]:
def is_far_from_1k(category, class_names_1k, thresh=0.1):
    #print('Checking if {} is far from ImageNet-1k'.format(category))
    obj1 = wn.synsets(category)[0]
    for some_class in class_names_1k:
        if some_class:
            obj2 = wn.synsets(some_class)[0]
            #if obj1.path_similarity(obj2) > thresh: 
            if obj1.lch_similarity(obj2) > thresh:
                #print('Nope')
                return False
    #print('Yes!')
    return True

Generate random synset_ids from synset_ids_21k which are not in synset_ids_1k and with minimum path distance to all 1k classes in ImageNet-1k.

In [50]:
nr_ood_classes = len(synset_ids_21k) - len(synset_ids_1k) # 200 # int(sys.argv[1])
ood_synset_ids_str = ''
nr_ood = 0

i = 0
start = time.time()
while i < nr_ood_classes:
    # random_idx = random.randint(0, max_id)
    # random_synset_id = synset_ids_21k[random_idx]
    # random_synset_name = class_names_21k[random_idx]
    ood_synset_id = synset_ids_21k[i]
    ood_class_name = class_names_21k[i]
    is_far_from_color = wn.synsets(ood_class_name)[0].lch_similarity(wn.synsets('color')[0]) < 1.5
    if ood_synset_id not in synset_ids_1k and is_far_from_1k(ood_class_name, class_names_1k, thresh=1.5) and is_far_from_color:
        ood_synset_ids_str += ood_synset_id + ' '
        nr_ood += 1
    i += 1
    if i % 2000 == 0:
        print(i)

end = time.time()
print('Done generating {} OOD classes after {:.2f} seconds!'.format(nr_ood_classes, end-start))

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
Done generating 20841 OOD classes after 291.15 seconds!


In [51]:
nr_ood

314

### 3. Generate in-distribution (ID) synset_ids 

Generate random synset_ids from ImageNet-1k. This will be the in-distribution data used.

In [13]:
nr_id_classes = 300 #int(sys.argv[2])
id_synset_ids_str = ''
max_id = len(synset_ids_1k)

i = 0
while i < nr_id_classes:
    random_synset_id = synset_ids_1k[random.randint(0, max_id)]
    id_synset_ids_str += random_synset_id + ' '
    i += 1

In [45]:
# Example of how path similarity works.
# 1
# obj1 = wn.synsets('color')[0]
# obj2 = wn.synsets('french_polish')[0]
# print('The similarity between {} and {} is: {}'.format('color', 'greenish_blue', obj1.lch_similarity(obj2)))
wn.synsets(ood_class_name)[0].lch_similarity(wn.synsets('color')[0]) < 1.5

True

In [38]:
# Example of how path similarity works.
# 1
obj1 = wn.synsets('ambulance')[0]
obj2 = wn.synsets('dog')[0]
print('The similarity between {} and {} is: {}'.format('ambulance', 'dog', obj1.lch_similarity(obj2)))
# 2
obj1 = wn.synsets('cat')[0]
obj2 = wn.synsets('kitty')[0]
print('The similarity between {} and {} is: {}'.format('cat', 'kitty', obj1.lch_similarity(obj2)))
# 3
obj1 = wn.synsets('cat')[0]
obj2 = wn.synsets('human')[0]
print('The similarity between {} and {} is: {}'.format('cat', 'human', obj1.lch_similarity(obj2)))
# 4
obj1 = wn.synsets('cat_food')[0]
obj2 = wn.synsets('dog_food')[0]
print('The similarity between {} and {} is: {}'.format('cat_food', 'dog_food', obj1.lch_similarity(obj2)))
# 5
obj1 = wn.synsets('white_rice')[0]
obj2 = wn.synsets('rice')[0]
print('The similarity between {} and {} is: {}'.format('white_rice', 'rice', obj1.lch_similarity(obj2)))

The similarity between ambulance and dog is: 0.9985288301111273
The similarity between cat and kitty is: 0.5020919437972361
The similarity between cat and human is: 1.6916760106710724
The similarity between cat_food and dog_food is: 2.538973871058276
The similarity between white_rice and rice is: 2.9444389791664407


In [29]:
print(f'This {nr_ood} classes below have similarity of max 0.1:')
ood_synset_ids_str

This 66 classes below have similarity of max 0.1:


'n01557185 n01848123 n01910252 n01915811 n02122298 n02430830 n02456962 n02708885 n02917521 n02982599 n03334912 n03349296 n03395256 n03498536 n03524976 n03566860 n03573848 n03683457 n03738472 n03846100 n03846234 n03856335 n03885194 n03901750 n04082562 n04083113 n04165675 n04368109 n04394031 n04960582 n04962062 n04963111 n04963307 n04963588 n04963740 n04964001 n04964799 n04965451 n04966941 n04967561 n04967674 n04967801 n04967882 n04968139 n04968749 n04969242 n04969540 n04969798 n04969952 n04970631 n04972801 n04973291 n04973386 n04973585 n04974145 n04976319 n04979002 n04979307 n07895435 n07914128 n10097477 n10117851 n10456696 n10498699 n11674332 n12770529 '

In [22]:
from collections import Counter
test = ood_synset_ids_str.split(' ')


Counter(test).keys() # equals to list(set(words))
Counter(test).values()

dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])